# Часть 2. Ситуационная (анализ проблем)

Ответы оформлены в виде структурированного конспекта с примером кода на Playwright.


## 1. Парсер перестал возвращать данные

### План диагностики
1. **Проверить логи и статус‑коды**
   - Ошибки подключения, таймауты, 403/429/500.
   - Количество успешно распарсенных записей (0, мало, нестабильно).
2. **Ручная проверка сайта в браузере**
   - Открыть тот же URL.
   - Проверить наличие CAPTCHA, логина, редиректов, технических страниц.
3. **Проверить инфраструктуру и окружение**
   - Интернет/прокси/VPN.
   - Токены, куки, доступ к аккаунту.
   - Недавние обновления библиотек, кода, деплой.
4. **Проанализировать ответ сайта**
   - Логирование сырого HTML/JSON, структуры данных.
   - Сравнение текущей разметки/структуры с предыдущей версией.
5. **Проверить селекторы и логику парсинга**
   - `locator(...).count()` в Playwright.
   - Актуальность CSS/XPath селекторов.
   - Логику фильтрации (не отбрасываем ли всё).
6. **Проверить запись в базу**
   - Ошибки вставки, ограничения UNIQUE/NOT NULL.
   - Конфликты типов и формат полей.

### Возможные причины
- Изменение верстки, CSS‑классов, DOM‑структуры.
- Переезд витрины на другой URL или в другой XHR/GraphQL.
- Появление или ужесточение антибот‑защиты (CAPTCHA, 403, 429).
- Истечение токенов, смена схемы авторизации.
- Регресс в коде парсера или изменённые настройки инфраструктуры.


## 2. Сайт начал блокировать запросы (CAPTCHA, 403, 429)

### Стратегии от простых к сложным

#### 2.1. Снижение нагрузки
- Уменьшить частоту запросов, добавить рандомные задержки.
- Снизить количество параллельных потоков.
- Кэшировать уже собранные данные.

#### 2.2. Эмуляция реального пользователя
- Использовать Playwright/Selenium вместо `requests`.
- Подставлять реалистичный `User-Agent`, `Accept-Language`, `Referer` и прочие заголовки.
- Имитировать нормальное поведение: скролл, переходы, без “телепортации” по URL.

#### 2.3. Работа с сессиями и авторизацией
- Использовать **persistent context** (постоянный профиль браузера):
  - один раз вручную залогиниться,
  - сохранить cookies/localStorage,
  - дальше запускать парсер уже как «живой пользователь».

#### 2.4. Обработка лимитов и ошибок
- Реализовать экспоненциальный backoff при 429/503.
- Читать `Retry-After`, если он есть.
- Отдельно логировать все HTTP‑ошибки и анализировать частоту.

#### 2.5. IP‑ротация и прокси (в рамках правил сайта)
- Использовать пул прокси и распределять запросы.
- Не стрелять много с одного IP.
- Использовать качественные (резидентские) прокси, если это необходимо.

#### 2.6. Изменение подхода
- Перейти на внутренние JSON/XHR/GraphQL API, если это возможно и разрешено.
- Минимизировать объём данных, не грузить лишние ресурсы.
- В идеале — договориться об официальном API / партнёрстве.

### Технологии и подходы для работы с CAPTCHA
- **Playwright/Selenium + реальный браузер**: корректный JS‑рендеринг и поведение.
- **Persistent context / сохранённый профиль**: повторное использование залогиненной сессии.
- **Ручное решение CAPTCHA** при отладке (`page.pause()` / паузы в нужных местах).
- **Сервисы антикапчи** (rucaptcha/2captcha и т.п.) — как крайняя мера.
- **IP‑ротация и контроль частоты запросов**.


## Пример рабочего кода: Playwright + сохранённый профиль (Ozon Seller)

Пример использования `launch_persistent_context`, когда профиль сессии (куки, localStorage)
хранится на диске и переиспользуется при каждом запуске парсера.


In [ ]:
from playwright.sync_api import sync_playwright


def login_with_saved_session() -> None:
    """Запуск браузера с сохранённым профилем и открытие защищённой страницы.

    Путь `user_data_path` указывает на директорию с профилем пользователя.
    Если профиля нет, он будет создан, можно залогиниться вручную,
    после чего последующие запуски будут использовать сохранённую сессию.
    """
    user_data_path = r"C:\\Users\\m.troshin\\PycharmProjects\\Pp\\get_ozon_competition_position\\data_user"

    with sync_playwright() as p:
        browser = p.chromium.launch_persistent_context(
            user_data_dir=user_data_path,
            headless=False,      # показываем окно браузера
            channel="chrome",   # используем реальный Chrome
        )

        # Берём первую открытую вкладку или создаём новую
        page = browser.pages[0] if browser.pages else browser.new_page()

        url = "https://seller.ozon.ru/app/analytics/what-to-sell/competitive-position"
        print(f"Открываю сайт: {url}")
        page.goto(url, wait_until="load")

        # Пауза, чтобы визуально проверить результат
        page.wait_for_timeout(5000)

        browser.close()
        print("Браузер закрыт.")


if __name__ == "__main__":
    login_with_saved_session()
